In [2]:
import pandas as pd
import gurobipy as gb
from gurobipy import *


In [5]:
corn_df = pd.read_excel('final_constraints.xlsx',sheet_name='Corn')
coef =[corn_df['nebraska_yield_coef'].to_list()]
coef.append(corn_df['illinois_yield_coef'].to_list())
coef.append(corn_df['iowa_yield_coef'].values.tolist())
corn_restrictions = pd.read_csv('corn_herbicide_restriction.csv')
restrictions = pd.read_csv('corn_herbicide_restriction_2.csv')
restrictions.rename(columns={'Unnamed: 0':'herbicide'},inplace=True)

In [44]:
# Parameters
# ---------------------------------------------------------------------------------------------
corn_state_name = ['Nebraska', 'Illinois', 'Iowa']
baseline = [178.5, 178, 184.9]
yield_minimum = [181, 201, 202]  # yield >= baseline + each herbicide * coef
num_weeks = 28
corn_herbicides = corn_df['herbicide_name'].unique()
corn_before_lb = corn_restrictions[corn_restrictions['type'] == 'before'][[
    'name', 'lower']]
corn_before_ub = corn_restrictions[corn_restrictions['type'] == 'before'][[
    'name', 'upper']]

num_states = len(corn_state_name)
num_herbicides_corn = len(corn_herbicides)
Range_herbicides = range(num_herbicides_corn)
Range_weeks = range(num_weeks)
Range_preplant = range(0, 4)
Range_atplant = range(4, 19)
Range_postplant = range(19, 28)
herbicide_names = restrictions['herbicide'].values

M = 1e7
eps = 1e-5

model = gb.Model('Herbicide Minimization for Corn')

# Decision variables
# ---------------------------------------------------------------------------------------------
x = model.addVars(num_states, num_herbicides_corn, num_weeks, vtype=gb.GRB.CONTINUOUS, name=(
    '{} quantity used for week {} in {}'.format(i, j, k) for k in corn_state_name for i in corn_herbicides for j in range(1, num_weeks+1)))

z = model.addVars(num_states, num_herbicides_corn, num_weeks, vtype=gb.GRB.BINARY, name=(
    'prescence of {} for week {} in {}'.format(i, j, k) for k in corn_state_name for i in corn_herbicides for j in range(1, num_weeks+1)))

model.ModelSense = gb.GRB.MINIMIZE

# Objective functions
# ---------------------------------------------------------------------------------------------
state_usage = model.addVars(num_states, vtype=gb.GRB.CONTINUOUS, name=(
    '{} total herbicide usage'.format(i) for i in corn_state_name))

model.addConstrs((state_usage[k] == quicksum(x[k, i, j] * z[k, i, j] for j in range(num_weeks) for i in range(
    num_herbicides_corn)) for k in range(num_states)), name='total usage')

model.setObjectiveN(
    quicksum(state_usage[k] for k in range(num_states)), priority=1, index=0)

glyphosate_usage = model.addVars(num_states, vtype=gb.GRB.CONTINUOUS, name=(
    '{} glyphosate usage'.format(i) for i in corn_state_name))

model.addConstrs((glyphosate_usage[k] == quicksum(x[k, 8, j] * z[k, 8, j]
                 for j in range(num_weeks)) for k in range(num_states)), name='glyphosate usage')

model.setObjectiveN(
    quicksum(glyphosate_usage[k] for k in range(num_states)), priority=0, index=1)


# Constraints
# ---------------------------------------------------------------------------------------------

# 1. Each state has a minimum amount of herbicide used for ensuring production yield

model.addConstrs((baseline[k]+quicksum(quicksum(x[k, i, j] for j in range(num_weeks))*coef[k][i]
                 for i in Range_herbicides) >= yield_minimum[k] for k in range(num_states)), name='minimum yield')


# 2. Each state has an upper bound of herbicide used for before planting
# 3. Each state has an upper bound of herbicide used for at planting
# 4. Each state has an upper bound of herbicide used for post planting
for i in range(herbicide_names.shape[0]):
    herbicide = herbicide_names[i]
    row = restrictions[restrictions['herbicide'] == herbicide]
    model.addConstrs((x[k, i, j]*z[k, i, j] * row['before'].values[0] <= row['before_ub'].values[0]
                     for j in Range_preplant for k in range(num_states)), name='before upper bound')

    model.addConstrs((x[k, i, j]*z[k, i, j] * row['during'].values[0] <= row['during_ub'].values[0]
                      for j in Range_atplant for k in range(num_states)), name='at planting upper bound')

    model.addConstrs((x[k, i, j]*z[k, i, j] * row['after'].values[0] <= row['after_ub'].values[0]
                      for j in Range_postplant for k in range(num_states)), name='post planting upper bound')


# 5. Special Quantity Restrictions
# 2,4-D W0-W18 at <= 0.41 kg/acre (0.855pt/acre) (1 L/ha)
model.addConstrs((quicksum(x[k, 0, j]*z[k, 0, j] for j in range(18)) <= 0.41
                 for k in range(num_states)), name='2,4-D upper bound')

# alachlor W4-W8 at 2.05 kg/acre (4.28 pt/acre) (5 L/ha)
model.addConstrs((quicksum(x[k, 1, j]*z[k, 1, j] for j in range(3, 8)) <= 2.05
                 for k in range(num_states)), name='alachlor upper bound')

# atrazine W0-W3 <= 1.814368kg/acre/year, W4-W8 at <= 1.91kg/acre (4 pt/acre/year)
model.addConstrs((quicksum(x[k, 2, j]*z[k, 2, j] for j in range(4)) <= 1.814368
                 for k in range(num_states)), name='atrazine upper bound')
model.addConstrs((quicksum(x[k, 2, j]*z[k, 2, j] for j in range(3, 8)) <= 1.91
                  for k in range(num_states)), name='atrazine upper bound')

# bromoxynil from W8 - W17 0.4 - 0.49 kg/acre
model.addConstrs((quicksum(x[k, 3, j]*z[k, 3, j] for j in range(7, 17)) <= 0.49
                 for k in range(num_states)), name='bromoxynil upper bound')
model.addConstrs((quicksum(x[k, 3, j]*z[k, 3, j] for j in range(7, 17)) >= 0.4
                  for k in range(num_states)), name='bromoxynil lower bound')

# dicamba W4-W6 at 0.506kg/acre (1.25 L/ha)
model.addConstrs((quicksum(x[k, 4, j]*z[k, 4, j] for j in range(3, 6)) == 0.506
                 for k in range(num_states)), name='dicamba upper bound')

# diflufenzoapyr before 2W at <= 0.17kg/acre (6 ozs/acre)
model.addConstrs((quicksum(x[k, 5, j]*z[k, 5, j] for j in range(2)) <= 0.17
                 for k in range(num_states)), name='diflufenzoapyr upper bound')

# diflufenzoapyr W4-W7 at <= 0.23kg/acre (8 ozs/acre)
model.addConstrs((quicksum(x[k, 5, j]*z[k, 5, j] for j in range(3, 7)) <= 0.23
                 for k in range(num_states)), name='diflufenzoapyr upper bound')

# diflufenzoapyr W8-W12 at <= 0.11kg/acre (4 ozs/acre)
model.addConstrs((quicksum(x[k, 5, j]*z[k, 5, j] for j in range(7, 12)) <= 0.11
                 for k in range(num_states)), name='diflufenzoapyr upper bound')

# eptc W0-W3 <=2.784kg/acre (7.33 pt/acre)
model.addConstrs((quicksum(x[k, 6, j]*z[k, 6, j] for j in range(4)) <= 2.784
                 for k in range(num_states)), name='eptc upper bound')

# eptc W4-W5 <=1.804kg/acre (3.978 lb/acre)
model.addConstrs((quicksum(x[k, 6, j]*z[k, 6, j] for j in range(3, 5)) <= 1.804
                 for k in range(num_states)), name='eptc upper bound')

# foramsulfuron before W8 <= 0.04082 kg/acre per year
model.addConstrs((quicksum(x[k, 7, j]*z[k, 7, j] for j in range(7)) <= 0.04082
                 for k in range(num_states)), name='foramsulfuron upper bound')

# glyphosate last application <W19 <=2.0581737kg/acre/year
model.addConstrs((quicksum(x[k, 8, j]*z[k, 8, j] for j in range(18, 28)) <= 2.0581737
                 for k in range(num_states)), name='glyphosate upper bound')

# halosulfron <=0.07541 kg/acre/year <=2 application/year, do not exceed per time
model.addConstrs((x[k, 9, j] <= 0.07541 for j in range(28)
                 for k in range(num_states)), name='halosulfron upper bound')

# nicosulfron <=0.0709764kg/acre
model.addConstrs((quicksum(x[k, 10, j]*z[k, 10, j] for j in range(28)) <= 0.0709764
                 for k in range(num_states)), name='nicosulfron upper bound')

# paraquat none

# pendimethalin none


# 6. Presence restriction
# 2,4-D 1 treatment per year
model.addConstrs((quicksum(z[k, 0, j] for j in range(28)) <= 1
                 for k in range(num_states)), name='2,4-D presence restriction')

# alachlor from W14 stop using
model.addConstrs((quicksum(z[k, 1, j] for j in range(13, 28)) == 0
                 for k in range(num_states)), name='alachlor presence restriction')

# atrazine none

# bromoxynil Minimum re-treatment interval for the second application is 21 days
bromoxynil_treat1 = model.addVars(
    num_states, vtype=GRB.INTEGER, lb=0, ub=24, name="week of the first treatment of bromoxynil")
bromoxynil_treat2 = model.addVars(
    num_states, vtype=GRB.INTEGER, lb=3, ub=27, name="week of the second treatment of bromoxynil")
bromoxynil_treat_aux = model.addVars(
    num_states, num_weeks, vtype=GRB.INTEGER, name="auxiliary variable for bromoxynil")

model.addConstrs((bromoxynil_treat_aux[k, j] == z[k, 3, j]*(Range_weeks[j]+1)
                  for j in Range_weeks for k in range(num_states)), name='bromoxynil auxiliary variable')

model.addConstrs((bromoxynil_treat1[k] <= 3 + bromoxynil_treat2[k]
                  for k in range(num_states)), name='bromoxynil first treatment')

model.addConstrs((bromoxynil_treat2[k] == max_(bromoxynil_treat_aux[k, j]
                                               for j in Range_weeks) for k in range(num_states)), name='bromoxynil second treatment')

model.addConstrs((quicksum(z[k, 3, j] for j in Range_weeks) <= 2
                  for k in range(num_states)), name='bromoxynil presence restriction')

# dicamba from W16 do not use
model.addConstrs((quicksum(z[k, 4, j] for j in range(15, 28)) == 0
                 for k in range(num_states)), name='dicamba presence restriction')

# diflufenzoapyr from W16 do not use
model.addConstrs((quicksum(z[k, 5, j] for j in range(15, 28)) == 0
                 for k in range(num_states)), name='diflufenzoapyr presence restriction')

# foramsulfuron from W9 do not use
model.addConstrs((quicksum(z[k, 7, j] for j in range(8, 28)) == 0
                 for k in range(num_states)), name='foramsulfuron presence restriction')

# glyphosate from W19 do not use
model.addConstrs((quicksum(z[k, 8, j] for j in range(18, 28)) == 0
                 for k in range(num_states)), name='glyphosate presence restriction')

# halosulfuron <=2 applications per year
model.addConstrs((quicksum(z[k, 9, j] for j in range(28)) <= 2
                 for k in range(num_states)), name='halosulfuron presence restriction')

# nicosulfuron none

# paraquat <=3 applications per year, not use between W4 and W8
model.addConstrs((quicksum(z[k, 11, j] for j in range(28)) <= 3
                 for k in range(num_states)), name='paraquat presence restriction')
model.addConstrs((quicksum(z[k, 11, j] for j in range(3, 8)) == 0
                  for k in range(num_states)), name='paraquat presence restriction')

# pendimethalin <=1 applications per year
model.addConstrs((quicksum(z[k, 12, j] for j in range(28)) <= 1
                 for k in range(num_states)), name='pendimethalin presence restriction')

# 7. Mixing restriction
# 2,4-D do not mix with glyphosate
model.addConstrs((z[k, 0, j] + z[k, 8, j] <= 1
                 for j in range(28) for k in range(num_states)), name='2,4-D glyphosate mixing restriction')

# parquat do not mix with glyphosate
model.addConstrs((z[k, 11, j] + z[k, 8, j] <= 1
                 for j in range(28) for k in range(num_states)), name='parquat glyphosate mixing restriction')

# 8. Auxiliary variables
# force z=1 when x>0; x<=0, z could be anything
model.addConstrs((x[k, i, j] <= z[k, i, j]*M
                 for j in range(num_weeks) for i in Range_herbicides for k in range(num_states)), name='z upper bound')

# force z=0 when x<=0
model.addConstrs((x[k, i, j] >= z[k, i, j]*eps
                 for j in range(num_weeks) for i in Range_herbicides for k in range(num_states)), name='z lower bound')

# Solve
# ---------------------------------------------------------------------------------------------

model.params.LogToConsole = 0
model.optimize()

# Print results
# ---------------------------------------------------------------------------------------------

print('Herbicide usage: %g' % model.getObjective(0).getValue())
print("Glyphosate usage: %g" % model.getObjective(1).getValue())

print('-'*100)

# for v in model.getVars():
#     if v.x > 0:
#         print('%s %g' % (v.varName, v.x))

# print('-'*100)
# for k in range(num_states):
#     print(corn_state_name[k], "used for pre-planting", quicksum(x[k,
#           i, j].x for j in Range_preplant for i in Range_herbicides))
#     print(corn_state_name[k], "used for at planting", quicksum(
#         x[k, i, j].x for j in Range_atplant for i in Range_herbicides))
#     print(corn_state_name[k], "used for post-planting", quicksum(x[k,
#           i, j].x for j in Range_postplant for i in Range_herbicides))

# print('-'*100)
# print("Yield for each state: ")
# for k in range(num_states):
#     print(corn_state_name[k], baseline[k] + quicksum(x[k, i, j].x*coef[k][i]
#           for i in Range_herbicides for j in Range_weeks))

# print('-'*100)
# for k in range(num_states):
#     print('='*100)
#     print(corn_state_name[k])
#     for i in Range_herbicides:
#         print("Usage value", corn_herbicides[i], quicksum(
#             x[k, i, j].x for j in Range_weeks))
#         print("Indicator", corn_herbicides[i], quicksum(
#             z[k, i, j].x for j in Range_weeks))

# Save to csv
# ---------------------------------------------------------------------------------------------
result_df = []
for k in range(num_states):
    for i in Range_herbicides:
        for j in Range_weeks:
            result_df.append({'state': corn_state_name[k], 'herbicide': corn_herbicides[i],
                              'week': j+1, 'usage': x[k, i, j].x, 'indicator': z[k, i, j].x})

result_df = pd.DataFrame(result_df,
                         columns=['state', 'herbicide', 'week', 'usage', 'indicator'])


print('Total yield for each state:')
for k in range(num_states):
    print(
        f'{corn_state_name[k]} : {baseline[k] + quicksum(x[k, i, j].x*coef[k][i] for i in Range_herbicides for j in Range_weeks)}')

display(result_df)

print('Total herbicide usage by state:')
display(result_df.groupby('state')['usage'].sum().to_frame())

print('Total glyphosate usage by state:')
display(result_df[result_df['herbicide'] == 'glyphosate'].groupby(
    'state')['usage'].sum().to_frame())

result_df.to_csv('corn_result.csv', index=False)

Herbicide usage: 5.68495
Glyphosate usage: 0
----------------------------------------------------------------------------------------------------
Total yield for each state:
Nebraska : 181.235695
Illinois : 201.0
Iowa : 202.0


,state,herbicide,week,usage,indicator
0,Nebraska,"2,4-D",1,0.0,-0.0
1,Nebraska,"2,4-D",2,0.0,-0.0
2,Nebraska,"2,4-D",3,0.0,-0.0
3,Nebraska,"2,4-D",4,0.0,-0.0
4,Nebraska,"2,4-D",5,0.0,-0.0
...,...,...,...,...,...
1087,Iowa,PENDIMETHALIN,24,0.0,-0.0
1088,Iowa,PENDIMETHALIN,25,0.0,-0.0
1089,Iowa,PENDIMETHALIN,26,0.0,-0.0
1090,Iowa,PENDIMETHALIN,27,0.0,-0.0


Total herbicide usage by state:


,usage
state,
Illinois,3.548799
Iowa,1.230143
Nebraska,0.906010


Total glyphosate usage by state:


,usage
state,
